<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/agcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_addons
!pip install pydub

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_addons as tfa
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
from IPython.display import Audio

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
import os
import librosa
import numpy as np

# Specify the folder path
folder_path = '/content/drive/MyDrive/bass samples'

# Initialize an empty list to store the audio chunks
audio_chunks = []

# Iterate through all the files in the folder
for file_name in os.listdir(folder_path):
    # Check if the file is a WAV file
    if file_name.endswith('.wav'):
        # Construct the file path
        file_path = os.path.join(folder_path, file_name)

        # Load the audio file using librosa
        audio, sr = librosa.load(file_path, sr=None)

        # Step 3: Cut the audio files into 1-second chunks with 50% overlap

        # Calculate the length of each chunk (1 second)
        chunk_length = int(sr)

        # Calculate the overlap length (50% of chunk_length)
        overlap_length = int(chunk_length / 8)

        # Calculate the total number of chunks
        num_chunks = int((len(audio) - chunk_length) / overlap_length) + 1

        # Iterate through the audio and extract the chunks
        for i in range(num_chunks):
            # Calculate the starting and ending indices of the chunk
            start_idx = i * overlap_length
            end_idx = start_idx + chunk_length

            # Extract the chunk
            chunk = audio[start_idx:end_idx]

            # Step 4: Discard any chunks that are not 1 second long

            if len(chunk) == chunk_length:
                # Append the chunk to the list
                audio_chunks.append(chunk)

# Step 5: Store the chunks in a numpy array with the desired shape

# Convert the list of chunks to a numpy array
audio_array = np.array(audio_chunks)

# Reshape the array to have the shape (num of chunks, length of audio, 1)
audio_array = audio_array.reshape(len(audio_array), len(audio_array[0]), 1)

noise = np.random.uniform(low=-1.0, high=1.0, size=audio_array.shape)




In [5]:
x_train = noise
y_train = audio_array
print(x_train.shape)

(173, 44100, 1)


In [6]:


audio_data = audio_array[5].ravel()
# Play the audio within the Jupyter Notebook
Audio(data=audio_data, rate=sr)

In [35]:
def build_discriminator():
    input_series = layers.Input(shape=(None,1))


    # Convolutional layers
    x = layers.Conv1D(256, 2, strides=2 , padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_series)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv1D(128, 10, strides=1 , padding='same',dilation_rate=10, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv1D(128, 2, strides=2 , padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv1D(128, 2, strides=2 , padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)

    # Global pooling
    pooled_output = layers.GlobalAveragePooling1D()(x)

    # Dense layer
    dense_output = layers.Dense(64, activation='relu')(pooled_output)

    # Dense layer
    dense_output = layers.Dense(1, activation='sigmoid')(pooled_output)

    model = tf.keras.models.Model(inputs=input_series, outputs=dense_output)
    return model

discriminator = build_discriminator()
discriminator_copy = build_discriminator()
discriminator.summary()


Model: "model_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, None, 1)]         0         
                                                                 
 conv1d_105 (Conv1D)         (None, None, 256)         768       
                                                                 
 leaky_re_lu_102 (LeakyReLU)  (None, None, 256)        0         
                                                                 
 dropout_93 (Dropout)        (None, None, 256)         0         
                                                                 
 conv1d_106 (Conv1D)         (None, None, 128)         327808    
                                                                 
 leaky_re_lu_103 (LeakyReLU)  (None, None, 128)        0         
                                                                 
 dropout_94 (Dropout)        (None, None, 128)         0  

In [39]:
def build_generator():
    input_series = layers.Input(shape=(None,1))


    # Generate noise using TensorFlow
    noise = tf.random.normal(shape=tf.shape(input_series))

    # Concatenate noise with input_series
    input_concat = tf.keras.layers.Concatenate()([input_series, noise])

    # Convolutional layers
    x = layers.Conv1D(256, 2, strides=2 , padding='same')(input_concat)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv1D(256, 5, strides=1 , padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)


    # Convolutional layers for output
    x = layers.Conv1D(256, 2, strides=1, padding='same')(x)
    x = layers.LeakyReLU()(x)


    # Conv1DTranspose layer for upsampling
    x = layers.Conv1DTranspose(1, 2, strides=2, padding='same',activation='tanh')(x)


    model = tf.keras.models.Model(inputs=input_series, outputs=x)
    return model

generator = build_generator()
generator.summary()


Model: "model_38"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_42 (InputLayer)          [(None, None, 1)]    0           []                               
                                                                                                  
 tf.compat.v1.shape_12 (TFOpLam  (3,)                0           ['input_42[0][0]']               
 bda)                                                                                             
                                                                                                  
 tf.random.normal_12 (TFOpLambd  (None, None, 1)     0           ['tf.compat.v1.shape_12[0][0]']  
 a)                                                                                               
                                                                                           

In [42]:
# Compile models
generator_optimizer = tf.keras.optimizers.Adam(0.00004)
discriminator_optimizer = tf.keras.optimizers.Adam(0.00001)
copy_optimizer = tf.keras.optimizers.Adam(0.00004)

#generator_optimizer = tf.keras.optimizers.experimental.SGD(1e-4)
#discriminator_optimizer = tf.keras.optimizers.experimental.SGD(1e-4)

In [43]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)



In [44]:
def print_img(generator_model):
    # Generate and save sample images
    noise = tf.random.normal([10, 100])
    sampled_labels = tf.constant([[i % 10] for i in range(10)], dtype=tf.int32)
    generated_images = generator_model.predict([noise, sampled_labels])
    fig, axs = plt.subplots(1, 10, figsize=(10, 10))
    for i in range(10):
        axs[i].imshow(generated_images[i], cmap="gray")
        axs[i].axis("off")
    plt.show()

In [ ]:
#@tf.function
def train_step(target_audios):

    batch_size = target_audios.shape[0]
    noise = tf.random.normal(shape=target_audios.shape)
    generated_audio = generator(noise, training=True)



    with tf.GradientTape() as disc_tape:

      real_output = discriminator([target_audios], training=True)
      fake_output = discriminator([generated_audio], training=True)

      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


    # Get the weights of model1
    weights = discriminator.get_weights()

    # Set the weights of model2 to be the same as model1
    discriminator_copy.set_weights(weights)

    for i in range(5):
      with tf.GradientTape() as disc_copy_tape:
        noise = tf.random.normal(shape=target_audios.shape)
        generated_audio = generator(noise, training=True)

        real_output = discriminator_copy([target_audios], training=True)
        fake_output = discriminator_copy([generated_audio], training=True)

        disc_loss = discriminator_loss(real_output, fake_output)

      gradients_of_discriminator = disc_copy_tape.gradient(disc_loss, discriminator_copy.trainable_variables)
      copy_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator_copy.trainable_variables))



    with tf.GradientTape() as gen_tape:
      noise = tf.random.normal(shape=target_audios.shape)
      generated_audio = generator(noise, training=True)
      fake_output = discriminator_copy([generated_audio], training=True)
      gen_loss = generator_loss(fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))


    tf.print("disc_loss",disc_loss,'gen_loss',gen_loss)


def train(generator, discriminator
          , epochs, batch_size):
    for epoch in range(epochs):
        for batch in range(len(x_train) // batch_size):
            #images = x_train[batch * batch_size: (batch+1) * batch_size]
            target_audios = y_train[batch * batch_size: (batch+1) * batch_size]

            train_step(target_audios)
        '''
        # Output training progress
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}")
            print_img(generator)
        '''

# Train the GAN
EPOCHS = 2000000
BATCH_SIZE = 10
num_unrolling_steps = 5  # Set the desired number of unrolling steps
train(generator, discriminator, EPOCHS, BATCH_SIZE)